In [88]:
#def init():
    #CHANGE THIS TO YOUR TRAIN FOLDER
image_folder = 'C:/Study/Semester2/Machine Learning/ML_images/smaller' 
#image_folder = 'C:/Users/danie/Desktop/Training_ML'

TOO_SMALL = 200000
SMALL = 500000
MEDIUM = 1200000
LARGE = 2100000
    
X_train = []
Y_train = []
counter = 1
    
img_height = 1888 #Usually 128. this might change depending on Annaya & Danielle's input 
img_width = 1888 #Usually 128. this might change depending on Annaya & Danielle's input 
epochs = 10 #Start with 1, and increase to 10, 100, 500, 1000 and 3000. 11 is the recommended number of runs through the training dataset. We will probably have to tune this. 
#total_training = total number of training data len()
batch_size = 1 #test different ones from 1 to 2, 5, 10, etc
total_images = 1

In [89]:
#pip install opencv-python

In [90]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage
import cv2
import glob
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, UpSampling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage import data, io,color
from skimage.io import imread_collection #loads a collection of images
from skimage.io import imread, imshow
from pathlib import Path

from sklearn.decomposition import PCA

import matplotlib as mpl
import matplotlib.pyplot as plt

In [91]:
#Define sizes of images based on their resolution
#Image shape and resolution

#TODO make the IMG sizes make sense


def shape(img):
    size = 0
    resolution = img.shape[0] * img.shape[1]
    if resolution <= TOO_SMALL:
        size = 0
    elif resolution <= SMALL:
        size = 1
    elif resolution <= MEDIUM:
        size = 2
    elif resolution <= LARGE:
        size = 3
    else:
        size = 6
    return size

In [92]:
def create_dataframe(image_folder):
    images = Path(image_folder).glob('*.jpg')

    list_of_files = []
    for image in images:
        list_of_files.append(str(image))

    df = pd. DataFrame(list_of_files, columns = ["image_path"])
    df_length = len(df.index)
    print("processing", df_length, "photos for training")
    
    height= []
    width = []
    resolution = []
    
    for image_path in df['image_path']:
        img = cv2.imread(str(image_path))
        height.append(img.shape[0])
        width.append(img.shape[1])
        
    df["height"] = height
    df["width"] = width
    df["resolution"] = df["height"] * df["width"]
    
    df = df[df.resolution < LARGE]
    
    ll = df_length - len(df.index)
    print("dropped ", ll, "photos because they were too large to process")
    
    df = df[df.resolution > TOO_SMALL]
    
    ss = df_length - len(df.index) - ll
    print("dropped ", ss, "photos because they were too small to process")
    df.head()
    
    return df

In [93]:
#PCA rgb then pass to prep_img as an rgb image

def pca_rgb(imgBGR, size): #imgRGB, size
    b,g,r = cv2.split(imgBGR)
    list_channels = [b,g,r]
    
    #check size first
    if size ==1:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 100)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)    

    elif size == 2:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 300)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)
    else:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 500)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)
        
    RGB_compressed = np.float32(RGB_compressed)
        
    return RGB_compressed

In [94]:
# image padding

def rgb_pad(image):
    
    height = image.shape[0]
    width = image.shape[1]

    WHITE = [255,255,255] 
    #MAX = 1888
    MAX_h = 750
    MAX_w = 600
    
    if height % 2 == 0:
        top = int((MAX_h - height)/ 2)
        bottom = int((MAX_h - height)/ 2)
    else: 
        top = int((MAX_h - (height-1))/ 2)
        bottom = int(((MAX_h - (height-1))/ 2)-1)

    if width % 2 == 0:
        left = int((MAX_w - width)/ 2)
        right = int((MAX_w - width)/ 2)
    else: 
        left = int((MAX_w - (width-1))/ 2)
        right = int(((MAX_w - (width-1))/2)-1)
    
    image = cv2.copyMakeBorder(image, top, bottom, left, right,
    cv2.BORDER_CONSTANT,value=WHITE)

    return image

In [95]:
#Debugged function
def prep_img(img_path):
    imgRGB = img_to_array(load_img(img_path))
    imgRGB = np.array(imgRGB, dtype=float)

    
    #check the image category by 'shape' function
    image_size_category = shape(imgRGB) 
    #print("image size determined!")

    if image_size_category <1:
        raise Exception("Your image is too small to be used")
    elif image_size_category >4:
        raise Exception("Your image is too big to be used")

    #print("image PCA started!")
 
    RGB_compressed = pca_rgb(imgRGB, image_size_category) 
    print('RGB_compressed:', np.min(RGB_compressed), np.max(RGB_compressed))
    #plt.figure(figsize=(10, 10))
    #plt.imshow(RGB_compressed)
    
    print("image is padded")
    padded_image = rgb_pad(RGB_compressed)
  
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image) # show after rgb compressed after padded
    
    #image_pca = np.float32((1.0/255)*image_pca) # #The rgb_compressed come out in RGB scaled data already
    print("image converted to LAB")
    padded_LAB = cv2.cvtColor(padded_image, cv2.COLOR_RGB2Lab) #convert padded RGB compressed to LAB
    print('Padded LAB:', np.min(padded_LAB), np.max(padded_LAB))
    
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image, cmap = plt.cm.gray) #show after convert padded rgb compressed to RGB with the function
  
    #separate L and AB color channels
    Y = padded_LAB[:, :,1:]
    Y = Y / 128 #comment out when show predicted
    print('A normalized:', np.min(Y[:,:,0]), np.max(Y[:,:,0]))
    print('B normalized:', np.min(Y[:,:,0]), np.max(Y[:,:,0]))
    
    X = padded_LAB[:, :, 0] 
    print('X:', np.min(X), np.max(X))
    #plt.figure(figsize=(10, 10)) # show converted RGB to L channel 
    #plt.imshow(X)
    
    #this is in sample code and idk why
    
    X = X.reshape(1, 1888, 1888, 1) #comment out when show predicted
    Y = Y.reshape(1, 1888, 1888, 2) #comment out when show predicted
    
    print("image processed")

    return Y, X 

In [96]:
def prep_img_no_pca(img_path):
    imgRGB = img_to_array(load_img(img_path)) #cv2.imread(img_path)
    print('imgRGB:', np.min(imgRGB), np.max(imgRGB))
    imgRGB = np.array(imgRGB, dtype=float)

    
    #check the image category by 'shape' function
    image_size_category = shape(imgRGB) #imgBGR
    print(image_size_category)
    #print("image size determined!")

    if image_size_category <1:
        raise Exception("Your image is too small to be used")
    elif image_size_category >4:
        raise Exception("Your image is too big to be used")

    #print("image PCA started!")
    #image_pca = pca_rgb(imgRGB, image_size_category) 
    #RGB_compressed = pca_rgb(imgRGB, image_size_category) #imgBGR, image_size_category
    #RGB_compressed = np.int32(RGB_compressed*255)
    #print('RGB_compressed:', np.min(RGB_compressed), np.max(RGB_compressed))
    #plt.figure(figsize=(10, 10))
    #plt.imshow(RGB_compressed)
    
    print("image is padded")
    #padded_image = rgb_pad(RGB_compressed)
    padded_image = rgb_pad(imgRGB)
    print('padded_image:', np.min(padded_image), np.max(padded_image))
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image) # show after rgb compressed after padded
    
    print("image converted to LAB")
   
    X_LAB = color.rgb2lab(1.0/255*padded_image)[:,:,0] #1.0/255*padded_image
    print('X_LAB:', np.min(X_LAB), np.max(X_LAB))
    Y_LAB = color.rgb2lab(1.0/255*padded_image)[:,:,1:] #1.0/255*padded_image
    Y_LAB = Y_LAB / 128
    print('Y_LAB:', np.min(Y_LAB), np.max(Y_LAB))
    
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image, cmap = plt.cm.gray) #show after convert padded rgb compressed to RGB with the function
    #imgLAB = np.clip(imgLAB, 0, 1)

    
    #separate L and AB color channels
    #Y = padded_LAB[:, :,1:]
    #Y = Y / 128 #comment out when show predicted
    #print('A:', np.min(Y[:,:,0]), np.max(Y[:,:,0]))
    #print('B:', np.min(Y[:,:,0]), np.max(Y[:,:,0]))
    
    #X = padded_LAB[:, :, 0] 
    #print('X:', np.min(X), np.max(X))
    #plt.figure(figsize=(10, 10)) # show converted RGB to L channel 
    #plt.imshow(X)
    
    #this is in sample code and idk why
    
    X = X_LAB.reshape(1, 750, 600, 1) #comment out when show predicted
    Y = Y_LAB.reshape(1, 750, 600, 2) #comment out when show predicted
    
    print("image processed")

    return Y, X 

In [97]:
def prep_dataframe(df):
    for index, row in df.iterrows():
        Y, X = prep_img(row["image_path"])
        Y_train.append(Y)
        X_train.append(X)

In [98]:
def prep_dataframe_no_pca(df):
    for index, row in df.iterrows():
        Y, X = prep_img_no_pca(row["image_path"])
        Y_train.append(Y)
        X_train.append(X)

# Use the code below to process ALL photos and feed them to the model!!!

In [99]:
df = create_dataframe(image_folder)
prep_dataframe_no_pca(df)



processing 121 photos for training
dropped  0 photos because they were too large to process
dropped  0 photos because they were too small to process
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.15513127599672444 0.3028308069753861
image processed
imgRGB: 18.0 255.0
1
image is padded
padded_image: 18.0 255.0
image converted to LAB
X_LAB: 10.91929841610147 100.0
Y_LAB: -0.3119397055590308 0.2483064112021963
image processed
imgRGB: 4.0 255.0
1
image is padded
padded_image: 4.0 255.0
image converted to LAB
X_LAB: 2.3650939351235962 100.0
Y_LAB: -0.24035910408412403 0.33285803735951336
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.16298837505321384 0.3641534348302679
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 4.599088300098778 99.82895934924873
Y_LAB: -0.45941285306387947 0.691604456340

Y_LAB: -0.1742408749369202 0.39134654868270585
image processed
imgRGB: 0.0 252.0
1
image is padded
padded_image: 0.0 252.0
image converted to LAB
X_LAB: 0.0 89.89181784773575
Y_LAB: -0.11894759127469923 0.34371852125689206
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.09163149211277415 0.2887723494995088
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.29501205258908314 100.0
Y_LAB: -0.23863937231245574 0.5543752386738079
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.039573654069084085 100.0
Y_LAB: -0.3862460230555937 0.5066250810469887
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.23323500629150473 100.0
Y_LAB: -0.12310556118656363 0.4458867379442942
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
i

Y_LAB: -0.2963564697816331 0.48329548053450244
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.5664558772864687 100.0
Y_LAB: -0.23047934078864393 0.46668656712297274
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.26975836970597866 0.21709844681164717
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 2.4179353089108417 100.0
Y_LAB: -0.21739904589740383 0.3417938878794732
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 99.92678386264787
Y_LAB: -0.30196113004349046 0.24393682482542037
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.34267872339963906 0.38033981997076094
image processed
imgRGB: 0.0 255.0
1
image is padded
padded_image: 0.0 255.0
image converted 

In [142]:
def mode(): 
    model2 = Sequential([
        #encoder
        Conv2D(128, 4, strides = 3, padding = "same", activation = "relu", input_shape= (750, 600, 1)), #testing shape changes
        Conv2D(256, 4, strides = 2, padding = "same", activation = "relu"), 
        Conv2D(256, 2, strides = 1, padding = "same", activation = "relu"), #filter = 256
        
        #decoder
        UpSampling2D(3),
        Conv2D(256, 2, padding = "same", activation = "relu"), 
        UpSampling2D(2),
        Conv2D(128, 2, padding = "same", activation = "relu"), 
        UpSampling2D(1), 
        Conv2D(2, 2, padding = "same", activation = "tanh"),
    ])
    model2.compile(optimizer ="adam", loss = "mse", metrics = ["accuracy"])
    return model2

In [143]:
model2 = mode()
model2.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_114 (Conv2D)         (None, 250, 200, 128)     2176      
                                                                 
 conv2d_115 (Conv2D)         (None, 125, 100, 256)     524544    
                                                                 
 conv2d_116 (Conv2D)         (None, 125, 100, 256)     262400    
                                                                 
 up_sampling2d_57 (UpSamplin  (None, 375, 300, 256)    0         
 g2D)                                                            
                                                                 
 conv2d_117 (Conv2D)         (None, 375, 300, 256)     262400    
                                                                 
 up_sampling2d_58 (UpSamplin  (None, 750, 600, 256)    0         
 g2D)                                                

In [126]:
len(X_train)

121

In [127]:
len(Y_train)

121

In [113]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train))

In [128]:
epochs = 1
history = model2.fit(train_data, batch_size = batch_size, epochs = epochs, verbose = 2)  # add validation_split = 0.2

121/121 - 385s - loss: 0.4050 - accuracy: 0.5212 - 385s/epoch - 3s/step


In [129]:
filename = 'my_model_tf3_E1'
model2.save(filename)

INFO:tensorflow:Assets written to: my_model_tf3_E1\assets


## Load model and test in Test notebook